In [27]:
# load csv
import pandas as pd
csv_path = r"https://docs.google.com/spreadsheets/d/1EvIFLGpsKvuOgdX6F9MYlGl9rCLjbKHPlE8srBDMwYo/edit?usp=sharing"

# Download file from Google Sheets
url = csv_path.replace('/edit?usp=sharing', '/export?format=csv')
df = pd.read_csv(url)

df.columns


Index(['id', 'date', 'writer', 'model', 'screen_size', 'chip', 'core', 'ram',
       'ssd', 'apple_care_plus', 'unused', 'price', 'price for statistics',
       'platform', 'url', 'title', 'content'],
      dtype='object')

In [28]:
import requests
item_csv_path = r"https://docs.google.com/spreadsheets/d/1eRkhdeDaD60IOajReYVRId28CM4IKQJiheL-yBWfzUw/edit?usp=sharing"
url = item_csv_path.replace('/edit?usp=sharing', '/export?format=xlsx')


item_df = pd.read_excel(url, sheet_name="Mac-Docs")

api_db = {}

for index, row in item_df.iterrows():
    value = (row["item id"], row["option id"])
    key = (row["item"], row["CPU"], row["CPU Core"], row["GPU Core"], row["RAM"], row["SSD"])
    print(key)
    api_db[key] = value


('Mac Mini', 'M1', 8, 8, 8, '256GB')
('Mac Mini', 'M1', 8, 8, 8, '512GB')
('Mac Mini', 'M1', 8, 8, 8, '1TB')
('Mac Mini', 'M1', 8, 8, 16, '256GB')
('Mac Mini', 'M1', 8, 8, 16, '512GB')
('Mac Mini', 'M1', 8, 8, 16, '1TB')
('Mac Mini', 'M2', 8, 10, 8, '256GB')
('Mac Mini', 'M2', 8, 10, 8, '512GB')
('Mac Mini', 'M2', 8, 10, 8, '1TB')
('Mac Mini', 'M2', 8, 10, 16, '256GB')
('Mac Mini', 'M2', 8, 10, 16, '512GB')
('Mac Mini', 'M2', 8, 10, 16, '1TB')
('Mac Mini', 'M2Pro', 10, 16, 16, '512GB')
('Mac Mini', 'M2Pro', 10, 16, 16, '1TB')
('Mac Mini', 'M2Pro', 10, 16, 32, '512GB')
('Mac Mini', 'M2Pro', 10, 16, 32, '1TB')
('Mac Mini', 'M2Pro', 12, 19, 16, '512GB')
('Mac Mini', 'M2Pro', 12, 19, 16, '1TB')
('Mac Mini', 'M2Pro', 12, 19, 32, '512GB')
('Mac Mini', 'M2Pro', 12, 19, 32, '1TB')
('Macbook Air', 'M1', 8, 7, 8, '256GB')
('Macbook Air', 'M1', 8, 7, 8, '512GB')
('Macbook Air', 'M1', 8, 7, 8, '1TB')
('Macbook Air', 'M1', 8, 7, 16, '256GB')
('Macbook Air', 'M1', 8, 7, 16, '512GB')
('Macbook Air', 

In [29]:
import datetime

start_date = datetime.datetime(2023, 3, 1)

def getKey(model, screen_size, chip, core, ram, ssd):
    if model == "Macbook Pro":
        _model = f"{model} {screen_size}"
    else:
        _model = model

    _cpu = chip.replace(" ", "")
    _cpu_core, _gpu_core = map(lambda x: int(x), core.replace(" ", "").split(","))
    _ram = int(ram.replace("GB", "").replace(" ", ""))
    _ssd = ssd.replace(" ", "")

    return _model, chip, _cpu_core, _gpu_core, _ram, _ssd

# models
datas = []
for week in range(9):
    end_date = start_date + datetime.timedelta(days=31)
    for model in df["model"].unique().tolist():
        model_data = df[df["model"] == model]
        for screen_size in model_data["screen_size"].unique():
            size_data = model_data[model_data["screen_size"] == screen_size]
            for chip in size_data["chip"].unique():
                chip_data = size_data[size_data["chip"] == chip]
                for core in chip_data["core"].unique():
                    core_data = chip_data[chip_data["core"] == core]
                    for ram in core_data["ram"].unique():
                        ram_data = core_data[core_data["ram"] == ram]
                        for ssd in ram_data["ssd"].unique():
                            ssd_data = ram_data[ram_data["ssd"] == ssd]

                            for unopened in [True, False]:
                                entity_list = ssd_data[(ssd_data["unused"] == unopened) & (start_date.strftime("%Y-%m-%d") <= ssd_data["date"]) & (ssd_data["date"] <= end_date.strftime("%Y-%m-%d"))]

                                if entity_list.size <= 1:
                                    continue

                                # 전처리
                                entity_list.loc[:, "price for statistics"] = entity_list["price for statistics"].str.replace(",", "").astype(int)

                                # 평균
                                mean = entity_list["price for statistics"].mean(numeric_only=True)

                                # 중앙 값
                                median = entity_list["price for statistics"].median()

                                # 표준 편차
                                std = entity_list["price for statistics"].std()

                                # 평균으로부터 표준편차 이내의 값 평균
                                mean_std = entity_list[(entity_list["price for statistics"] >= (mean - std)) & (entity_list["price for statistics"] <= (mean + std))]["price for statistics"].mean()

                                # 상위 0 ~ 20% 사이 값들의 평균
                                top_20_percentile = entity_list["price for statistics"].quantile(0.8)
                                top_20_percent_values = entity_list[entity_list["price for statistics"] >= top_20_percentile]
                                mean_20 = top_20_percent_values.mean(numeric_only=True)["price for statistics"]

                                # 하위 20% 평균
                                bottom_20_percentile = entity_list["price for statistics"].quantile(0.2)
                                bottom_20_percent_values = entity_list[entity_list["price for statistics"] <= bottom_20_percentile]
                                mean_80 = bottom_20_percent_values.mean(numeric_only=True)["price for statistics"]

                                key = getKey(model, screen_size, chip, core, ram, ssd)

                                if key not in api_db:
                                    print(f"key not found: {key}")
                                    continue
                                else:
                                    item_id, option_id = api_db[key]

                                if mean_80 == mean_20:
                                    mean_std = mean_20
                                    mean_80 = mean_20 = 0

                                datas.append({
                                    "item_id": item_id,
                                    "option_id": option_id,
                                    "start_date": start_date.strftime("%Y-%m-%d"),
                                    "end_date": end_date.strftime("%Y-%m-%d"),
                                    "mean_std": round(mean_std),
                                    "mean_20": round(mean_20),
                                    "mean_80": round(mean_80),
                                })

    start_date += datetime.timedelta(days=7)

# Save to CSV
df = pd.DataFrame(datas)
df.to_csv("data.csv", index=False)

key not found: ('Macbook Pro 14', 'M1Max', 10, 24, 16, '1TB')
key not found: ('Macbook Pro 14', 'M1Max', 10, 32, 16, '512GB')
key not found: ('Macbook Pro 14', 'M1Pro', 10, 16, 16, '2TB')
key not found: ('Macbook Pro 14', 'M1Max', 10, 24, 16, '1TB')
key not found: ('Macbook Pro 14', 'M1Max', 10, 32, 16, '512GB')
key not found: ('Macbook Pro 14', 'M2Max', 12, 30, 16, '1TB')
key not found: ('Macbook Pro 16', 'M1Max', 10, 32, 64, '4TB')
key not found: ('Macbook Pro 16', 'M2Max', 12, 30, 32, '1TB')
key not found: ('Macbook Pro 16', 'M2Max', 12, 38, 32, '1TB')


/var/folders/pd/nw98c7v10d5dhj5jdww513j80000gn/T/ipykernel_48799/928057084.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  entity_list.loc[:, "price for statistics"] = entity_list["price for statistics"].str.replace(",", "").astype(int)
/var/folders/pd/nw98c7v10d5dhj5jdww513j80000gn/T/ipykernel_48799/928057084.py:42: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  entity_list.loc[:, "price for statistics"] = entity_list["price for statistics"].str.replace(",", "").astype(int)
/var/folders/pd/nw98c7v10d5dhj5jdww513j80000g

key not found: ('Macbook Pro 14', 'M1Pro', 10, 16, 16, '2TB')

/var/folders/pd/nw98c7v10d5dhj5jdww513j80000gn/T/ipykernel_48799/928057084.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  entity_list.loc[:, "price for statistics"] = entity_list["price for statistics"].str.replace(",", "").astype(int)
/var/folders/pd/nw98c7v10d5dhj5jdww513j80000gn/T/ipykernel_48799/928057084.py:42: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  entity_list.loc[:, "price for statistics"] = entity_list["price for statistics"].str.replace(",", "").astype(int)
/var/folders/pd/nw98c7v10d5dhj5jdww513j80000g


key not found: ('Macbook Pro 14', 'M1Max', 10, 24, 16, '1TB')
key not found: ('Macbook Pro 14', 'M1Max', 10, 32, 16, '512GB')
key not found: ('Macbook Pro 14', 'M2Max', 12, 30, 16, '1TB')
key not found: ('Macbook Pro 16', 'M1Max', 10, 32, 64, '4TB')
key not found: ('Macbook Pro 16', 'M2Max', 12, 30, 32, '1TB')
key not found: ('Macbook Pro 16', 'M2Max', 12, 38, 32, '1TB')
key not found: ('Macbook Pro 14', 'M1Pro', 10, 16, 16, '2TB')
key not found: ('Macbook Pro 14', 'M1Max', 10, 24, 16, '1TB')
key not found: ('Macbook Pro 14', 'M1Max', 10, 32, 16, '512GB')
key not found: ('Macbook Pro 14', 'M2Max', 12, 30, 16, '1TB')
key not found: ('Macbook Pro 16', 'M2Pro', 12, 19, 64, '512GB')
key not found: ('Macbook Pro 16', 'M1Pro', 10, 16, 32, '2TB')
key not found: ('Macbook Pro 16', 'M1Max', 10, 32, 64, '4TB')
key not found: ('Macbook Pro 16', 'M1Max', 10, 32, 64, '2TB')


...

/var/folders/pd/nw98c7v10d5dhj5jdww513j80000gn/T/ipykernel_48799/928057084.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  entity_list.loc[:, "price for statistics"] = entity_list["price for statistics"].str.replace(",", "").astype(int)
/var/folders/pd/nw98c7v10d5dhj5jdww513j80000gn/T/ipykernel_48799/928057084.py:42: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  entity_list.loc[:, "price for statistics"] = entity_list["price for statistics"].str.replace(",", "").astype(int)
/var/folders/pd/nw98c7v10d5dhj5jdww513j80000g

key not found: ('Macbook Air', 'M2', 8, 10, 24, '512GB')
key not found: ('Macbook Air', 'M1', 8, 7, 16, '2TB')
key not found: ('Macbook Pro 14', 'M2Max', 12, 30, 32, '1TB')
key not found: ('Macbook Pro 16', 'M2Pro', 12, 19, 64, '512GB')
key not found: ('Macbook Pro 16', 'M1Pro', 10, 16, 32, '2TB')
key not found: ('Macbook Pro 16', 'M1Pro', 10, 16, 64, '512GB')
key not found: ('Macbook Pro 16', 'M1Max', 10, 32, 64, '2TB')
key not found: ('Macbook Pro 16', 'M1Max', 10, 32, 32, '2TB')
key not found: ('Macbook Pro 16', 'M1Max', 10, 24, 32, '2TB')
key not found: ('Macbook Pro 16', 'M1Max', 10, 24, 64, '2TB')
key not found: ('Macbook Pro 16', 'M2Max', 12, 30, 32, '1TB')
key not found: ('Macbook Pro 16', 'M2Max', 12, 38, 32, '1TB')
key not found: ('Macbook Pro 16', 'M2Max', 12, 38, 32, '1TB')
key not found: ('Macbook Air', 'M2', 8, 10, 24, '512GB')
key not found: ('Macbook Air', 'M1', 8, 7, 16, '2TB')


/var/folders/pd/nw98c7v10d5dhj5jdww513j80000gn/T/ipykernel_48799/928057084.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  entity_list.loc[:, "price for statistics"] = entity_list["price for statistics"].str.replace(",", "").astype(int)
/var/folders/pd/nw98c7v10d5dhj5jdww513j80000gn/T/ipykernel_48799/928057084.py:42: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  entity_list.loc[:, "price for statistics"] = entity_list["price for statistics"].str.replace(",", "").astype(int)
/var/folders/pd/nw98c7v10d5dhj5jdww513j80000g

key not found: ('Macbook Pro 13', 'M1Pro', 8, 8, 8, '256GB')
key not found: ('Macbook Pro 13', 'M1Pro', 8, 8, 8, '512GB')
key not found: ('Macbook Pro 13', 'M1Pro', 8, 8, 8, '512GB')
key not found: ('Macbook Pro 13', 'M1Pro', 8, 8, 16, '256GB')
key not found: ('Macbook Pro 13', 'M1Pro', 8, 8, 16, '512GB')
key not found: ('Macbook Pro 13', 'M2Pro', 8, 10, 16, '256GB')
key not found: ('Macbook Pro 13', 'M2Pro', 8, 10, 16, '512GB')
key not found: ('Macbook Pro 13', 'M2Pro', 8, 10, 24, '1TB')
key not found: ('Macbook Pro 14', 'M1Pro', 10, 16, 32, '2TB')
key not found: ('Macbook Pro 14', 'M1Pro', 10, 16, 32, '2TB')
key not found: ('Macbook Pro 14', 'M2Pro', 8, 14, 16, '512GB')
key not found: ('Macbook Pro 14', 'M2Max', 12, 30, 32, '1TB')
key not found: ('Macbook Pro 16', 'M1Pro', 10, 16, 32, '2TB')
key not found: ('Macbook Pro 16', 'M1Pro', 10, 16, 64, '512GB')
key not found: ('Macbook Pro 16', 'M1Max', 10, 32, 32, '2TB')
key not found: ('Macbook Pro 16', 'M1Max', 10, 24, 32, '2TB')
key not

/var/folders/pd/nw98c7v10d5dhj5jdww513j80000gn/T/ipykernel_48799/928057084.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  entity_list.loc[:, "price for statistics"] = entity_list["price for statistics"].str.replace(",", "").astype(int)
/var/folders/pd/nw98c7v10d5dhj5jdww513j80000gn/T/ipykernel_48799/928057084.py:42: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  entity_list.loc[:, "price for statistics"] = entity_list["price for statistics"].str.replace(",", "").astype(int)
/var/folders/pd/nw98c7v10d5dhj5jdww513j80000g

In [187]:
import time
# API Call
api_url_template = "https://asia-northeast3-macguider-44e7e.cloudfunctions.net/api/item/{}/option/{}"

# read csv
df = pd.read_csv("stats.csv")

for index, row in df.iterrows():
    item_id = row["item_id"]
    option_id = row["option_id"]
    date = row["end_date"]
    mean = row["mean_std"]
    high = row["mean_20"]
    low = row["mean_80"]
    unopened = row["unopened"]


    api_url = api_url_template.format(item_id, option_id)
    if unopened:
        api_url += "?unopened=true"
    else:
        api_url += "?unopened=false"

    print(api_url)

    data = {
        "date": date,
        "mid": int(mean),
        "high": int(high),
        "low": int(low),
    }


    print(f"{data}")
    # response = requests.post(api_url, json=data)
    # print(response.status_code)
    # print(response.text)
    # print()
    # time.sleep(1)


https://asia-northeast3-macguider-44e7e.cloudfunctions.net/api/item/1/option/1?unopened=true
{'date': '2023-04-05', 'mid': 0, 'high': 0, 'low': 0}
https://asia-northeast3-macguider-44e7e.cloudfunctions.net/api/item/1/option/1?unopened=true
{'date': '2023-04-12', 'mid': 0, 'high': 0, 'low': 0}
https://asia-northeast3-macguider-44e7e.cloudfunctions.net/api/item/1/option/1?unopened=true
{'date': '2023-04-19', 'mid': 0, 'high': 0, 'low': 0}
https://asia-northeast3-macguider-44e7e.cloudfunctions.net/api/item/1/option/1?unopened=true
{'date': '2023-04-26', 'mid': 0, 'high': 0, 'low': 0}
https://asia-northeast3-macguider-44e7e.cloudfunctions.net/api/item/1/option/1?unopened=false
{'date': '2023-04-05', 'mid': 540000, 'high': 700000, 'low': 530000}
https://asia-northeast3-macguider-44e7e.cloudfunctions.net/api/item/1/option/1?unopened=false
{'date': '2023-04-12', 'mid': 576000, 'high': 685000, 'low': 530000}
https://asia-northeast3-macguider-44e7e.cloudfunctions.net/api/item/1/option/1?unopene